# AIML CA2

## Import General Dependencies

In [ ]:
# Data Manipulation Dependencies
import numpy as np
import pandas as pd

# Graphing Dependencies
import matplotlib.pyplot as plt
import seaborn as sns

# Miscellaneous Dependencies
from typing import Union, List, Tuple, Dict, Callable
import warnings

In [ ]:
# Hide all warnings
warnings.filterwarnings(action='ignore')
from statsmodels.tools.sm_exceptions import ConvergenceWarning

# Part A > Time Series Regression

## Import Exclusive Dependencies

In [ ]:
# Time Series Dependencies
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose

## Import Data

In [ ]:
df = pd.read_csv('./data/train.csv')
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df.set_index(keys='Date', inplace=True)
df.sort_index(inplace=True)
df

## Data Cleaning

In [ ]:
df.groupby(by=[df.index.year, df.index.quarter]).count()

In [ ]:
def groupby_date(df: pd.DataFrame, freq: str = 'D'):
    df_tmp = df.copy()
    gbo = df_tmp.resample(rule=freq).mean()
    return gbo

In [ ]:
def drop_unreasonable_values(df: pd.DataFrame, gas: str):
    mask = df['Value'] > 0
    df_tmp = df[mask]
    dropped_rows = df[~mask]
    print(f'Dropped {dropped_rows.shape[0]} rows for {gas} (unreasonable)')
    return df_tmp

In [ ]:
def drop_outliers(df: pd.DataFrame, gas: str, band: float = 1.5):
    vals = df['Value'].values
    Q1 = np.quantile(vals, q=0.25)
    Q3 = np.quantile(vals, q=0.75)
    IQR = Q3 - Q1
    df_tmp = df[(df['Value'] > (Q1 - band * IQR)) & (df['Value'] < (Q3 + band * IQR))]
    print(f'Dropped {df.shape[0] - df_tmp.shape[0]} rows for {gas} (outlier)')
    return df_tmp

In [ ]:
def drop_sparse_data(df, gas: str, threshold: int = 20, freq: str = 'days'):
    from datetime import timedelta
    index = df.index
    x = 0
    drop_rows = []
    for i in df.index:
        for n in range(1, threshold + 1):
            if (i + timedelta(**{freq: n})) not in index:
                drop_rows.append(i)
                break
        x += 1
    print(f'Dropped {len(drop_rows)} rows for {gas} (sparse)')
    return df.drop(drop_rows, axis=0)

In [ ]:
def fill_missing_dates(df: pd.DataFrame, freq: str = 'D'):
    dummy_date_range = pd.date_range(start=df.index[0], end=df.index[-1], freq=freq)
    dummy_date_frame = pd.DataFrame(index=dummy_date_range, data=[df[date]['Value'] if np.isin(date, df.index) else np.nan for date in dummy_date_range])
    filled_df = pd.merge(left=df, right=dummy_date_frame, left_index=True, right_index=True, how='right').drop(columns=0)
    print(f'Inserted {filled_df.shape[0] - df.shape[0]} {df.shape} rows (filled)')
    return filled_df

In [ ]:
def impute_missing_values(df: pd.DataFrame, method: str = 'time'):
    return df.interpolate(method=method)

In [ ]:
def get_df_partitioned_by_gas(df: pd.DataFrame, resample_freq: str = 'D'):
    gas_dict = {}
    for gas in df['Gas'].unique():
        df_tmp = df[df['Gas'] == gas]
        # df_tmp = drop_unreasonable_values(df_tmp, gas=gas)
        df_tmp = drop_outliers(df_tmp, gas)
        # df_tmp = drop_sparse_data(df_tmp, gas=gas, threshold=3)
        df_tmp = fill_missing_dates(df=df_tmp, freq=resample_freq)
        df_tmp = impute_missing_values(df_tmp)
        df_tmp['Gas'] = [gas] * df_tmp.shape[0]
        gas_dict[gas] = df_tmp
    return gas_dict

df_partitioned = get_df_partitioned_by_gas(df)
df_partitioned['CO'].tail()

## Exploratory Data Analysis

In [ ]:
def get_gas_distributions(df: pd.DataFrame):
    counts = df.groupby(by='Gas').count().median(axis=1).astype(int)
    sns.barplot(x=counts.index, y=counts, palette='deep')

get_gas_distributions(df)

In [ ]:
def get_gas_value_distributions(df_dict: Dict[str, pd.DataFrame]):
    fig, ax = plt.subplots(nrows=2, ncols=2)
    for i, gas in enumerate(df_dict.keys()):
        current_ax = ax[i // 2, i % 2]
        current_ax.set_title(gas)
        sns.histplot(data=df_dict[gas]['Value'], kde=True, ax=current_ax)

get_gas_value_distributions(df_partitioned)

In [ ]:
def get_general_trends(df_dict: Dict[str, pd.DataFrame], ma_windows: List[int] = [1, 7, 30], kind: str = 'group'):
    cells = len(df_dict.keys())
    fig, ax = plt.subplots(nrows=cells // 2, ncols=cells // 2, figsize=(12, 8))
    for i, gas in enumerate(df_dict.keys()):
        df = df_dict[gas]
        for w in ma_windows:
            df_tmp = df.rolling(window=w).mean() if kind == 'ma' else groupby_date(df, freq=f'{w}D')
            ax[i // 2, i % 2].plot(df_tmp['Value'], label=f'{w}-day {kind}')
    ax[0, 0].legend()

get_general_trends(df_dict=df_partitioned, ma_windows=[1, 7, 30])

In [ ]:
def get_plots_by_gas(df_dict: pd.DataFrame):
    def get_plot(df: pd.DataFrame, ax):
        ax.plot(df['T'], label='T')
        ax.plot(df['RH'], label='RH')
        ax.plot([], [], color='g', label='Value')

        ax2 = ax.twinx()
        ax2.plot(df['Value'], color='green')
        # ax.set_xticklabels(labels=df.index.month)

    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))
    gg = [(0, 0), (0, 1), (1, 0), (1, 1)]
    
    for i, gas in enumerate(df_dict.keys()):
        current_ax = ax[(i // 2, i % 2)]
        df_tmp = df_dict[gas]
        get_plot(df_tmp, ax=current_ax)
        current_ax.set_title(gas)
        cc = current_ax.get_xticks()
    
    ax[0, 0].legend()
    fig.tight_layout()

get_plots_by_gas(df_dict=df_partitioned)

In [ ]:
def get_gas_means_and_medians(df):
    # fig, ax = plt.subplots(nrows=2, ncols=2)
    gb = df.groupby(by='Gas', as_index=False)
    mean_ = gb.mean()
    mean_['Type'] = ['Mean'] * mean_.shape[0]
    types = ['Mean', 'Median']
    comb_df = pd.DataFrame()
    for i, frame in enumerate([gb.mean(), gb.median()]):
        tmp_df = frame
        tmp_df['Type'] = types[i]
        comb_df = pd.concat(objs=(comb_df, tmp_df), axis=0)
    # print(comb_df)
    sns.barplot(data=comb_df, x='Gas', y='Value', hue='Type', palette='rainbow')
get_gas_means_and_medians(df)

In [ ]:
def get_corr(df_dict: Dict[str, pd.DataFrame]):
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10, 8))
    for i, gas in enumerate(df_dict.keys()):
        current_ax = ax[i // 2, i % 2]
        sns.heatmap(data=df_dict[gas].corr().abs(), annot=True, cmap='bone_r', vmin=0.0, vmax=1.0, ax=current_ax)
        current_ax.set_title(gas)

get_corr(df_partitioned)

## Algorithm Selection and Hyper-Parameter Tuning

### Tools

In [ ]:
def cf_summary(df: pd.DataFrame, kind: Union['pacf', 'acf'], threshold: float = 0.01, max_lag: int = 40, plot: bool = False):
    from statsmodels.tsa.stattools import pacf, acf
    from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
    corr_fn = pacf if kind == 'pacf' else acf
    plot_fn = plot_pacf if kind == 'pacf' else plot_acf
    lag_space = min(max_lag, df.count() // 2 - 1)
    corr_scores, conf_intvs = corr_fn(df, nlags=lag_space, alpha=threshold)
    lower_conf_bound = conf_intvs[:, 0] - corr_scores
    upper_conf_bound = conf_intvs[:, 1] - corr_scores
    t_values = np.where((corr_scores < lower_conf_bound) | (corr_scores > upper_conf_bound))[0][1:]
    best_t = t_values[np.argmax((np.abs(corr_scores[t_values])))] if len(t_values) > 0 else 0
    if plot:
        plot_fn(df, lags=lag_space, alpha=threshold, zero=False).get_axes()[0].set_title(f'possible t-values: {tuple(t_values)} Best t: {best_t}')
    t_dict = {}
    return t_values, best_t

In [ ]:
def get_seasonal_periods(df_dict: Dict[str, pd.DataFrame], plot: bool = False):
    value_dict = {}
    if plot:
        fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))
    for i, gas in enumerate(df_dict.keys()):
        df = df_dict[gas]
        seasonal_component = seasonal_decompose(df['Value']).seasonal
        _, m = cf_summary(seasonal_component, kind='acf')
        value_dict[gas] = m
        if plot:
            focus = seasonal_component['2004-04-01':'2004-04-20']
            current_ax = ax[i // 2, i % 2]
            current_ax.plot(focus.index.day, focus)
            current_ax.set_xticks(focus.index.day)
            current_ax.set_title(gas)
    return value_dict

get_seasonal_periods(df_partitioned)

In [ ]:
def test_stationarity(df: pd.DataFrame, cf_threshold: float, significance_level: float = 0.05, ax = None):
    from statsmodels.tsa.statespace.tools import diff
    p_value = adfuller(df['Value'])[1]
    p_values, best_p = cf_summary(diff(df['Value'], k_diff=0, k_seasonal_diff=1, seasonal_periods=7), kind='pacf', max_lag=6, threshold=cf_threshold)
    q_values, best_q = cf_summary(diff(df['Value'], k_diff=0, k_seasonal_diff=1, seasonal_periods=7), kind='acf', max_lag=6, threshold=cf_threshold)
    if ax is not None:
        decomposition = seasonal_decompose(df['Value'], model='additive')
        decomposition.plot(ax=ax)
    return p_value < significance_level, p_values, best_p, q_values, best_q

In [ ]:
def get_param_summary(df_dict: Dict[str, pd.DataFrame], threshold: float):
    summary_dict = {}
    for gas in df_dict.keys():
        stationary, p_values, best_p, q_values, best_q = test_stationarity(df_dict[gas], cf_threshold=threshold, significance_level=0.01)
        summary_dict[gas] = {
            'stationary': stationary,
            'p_values': p_values,
            'best_p': best_p,
            'q_values': q_values,
            'best_q': best_q
        }
    return summary_dict

get_param_summary(df_partitioned, threshold=0.005)

In [ ]:
# baseline prediction, since they are all stationary
# test_set = pd.read_csv(filepath_or_buffer='./data/test.csv', sep=',')
# pd.merge(left=test_set, right=df.groupby(by='Gas').mean(), left_on='Gas', right_index=True, how='inner')[['id', 'Value']].set_index(keys='id').to_csv('./out/using_mean.csv', sep=',')

In [ ]:
def get_params(df_dict: Dict[str, pd.DataFrame], threshold: float = 0.01):
    param_summary = get_param_summary(df_dict, threshold=threshold)
    orders = {}
    for i, gas in enumerate(df_dict.keys()):
        p = tuple(param_summary[gas]['p_values'])
        d = 0 if param_summary[gas]['stationary'] else 1
        q = tuple(param_summary[gas]['q_values'])
        orders[gas] = (p, d, q)
    return orders

get_params(df_dict=df_partitioned)

In [ ]:
# def partition_params(param_dict: Dict[str, Tuple], seasonal_dict: Dict[str, int]):
#     orders = {}
#     for gas in param_dict.keys():
#         order = param_dict[gas]
#         m = seasonal_dict[gas]

#         p = tuple([_p for _p in order[0] if _p % m > 0])
#         q = tuple([_q for _q in order[2] if _q % m > 0])

#         P = tuple([int(_p / m) for _p in order[0] if _p % m == 0])
#         Q = tuple([int(_q / m) for _q in order[2] if _q % m == 0])
        
#         seasonal_params = (P, 0, Q, m) if (len(P) + len(Q)) > 0 else None

#         orders[gas] = {
#             'trend_order': (p, order[1], q),
#             'seasonal_order': seasonal_params
#         }
#     return orders

# partition_params(get_params(df_dict=df_partitioned), get_seasonal_periods(df_partitioned))

### Testing

In [ ]:
def evaluate_params(df: pd.DataFrame, algo: Union['sarima', 'expsmooth'], init_kws: Dict, fit_kws: Dict, test_size: float = 0.75):
    from sklearn.metrics import mean_squared_error
    from statsmodels.tsa.holtwinters import ExponentialSmoothing
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', ConvergenceWarning)
        models = {}

        data = df['Value']
        partition_index = int(data.shape[0] * test_size)

        train = data[:partition_index]
        test = data[partition_index:]

        if algo == 'sarima':
            # model = SARIMAX(train, order=orders['trend_order'], seasonal_order=orders['seasonal_order']).fit()
            model = SARIMAX(endog=train, **init_kws).fit(**fit_kws)
        else:
            model = ExponentialSmoothing(endog=train, **init_kws).fit(**fit_kws)
        
        train_pred = model.predict(start=0, end=partition_index - 1)
        test_pred = model.forecast(steps=data.shape[0] - partition_index)
        # test_pred = model.predict(start=partition_index, end=data.shape[0] - 1)

        train_err = mean_squared_error(train[7:], train_pred[7:], squared=False)
        test_err = mean_squared_error(test, test_pred, squared=False)

        return {
            'aic': model.aic,
            'train_true': train,
            'train_pred': train_pred,
            'train_rmse': train_err,
            'test_true': test,
            'test_pred': test_pred,
            'test_rmse': test_err
        }

In [ ]:
def grid_search_sarima(df_dict: Dict[str, pd.DataFrame]):
    from numpy.linalg import LinAlgError
    gs_summary = {
        'CO': [],
        'NOx': [],
        'NMHC': [],
        'O3': []
    }
    thresholds = [0.005, 0.01]
    seasonal_params = {
        'P_space': [0, 1, 2, 4],
        'D_space': [0, 1],
        'Q_space': [0, 1, 2, 4]
    }
    seasonal_periods = get_seasonal_periods(df_dict=df_dict)
    for t in thresholds:
        trend_orders = get_params(df_dict=df_dict, threshold=t)
        for gas in df_dict.keys():
            T = trend_orders[gas]
            m = seasonal_periods[gas]
            # S_grid = seasonal_params[gas]
            S_grid = seasonal_params
            df = df_dict[gas]
            for P in S_grid['P_space']:
                for D in S_grid['D_space']:
                    for Q in S_grid['Q_space']:
                        try:
                            # results = evaluate_params(df, orders={'trend_order': T, 'seasonal_order': (P, D, Q, m)})
                            results = evaluate_params(df, algo='sarima', init_kws={'order': T, 'seasonal_order': (P, D, Q, m)})
                            gs_summary[gas].append({
                                'orders': (T, (P, D, Q, m)),
                                'aic': results['aic'],
                                'train_rmse': results['train_rmse'],
                                'test_rmse': results['test_rmse']
                            })
                        except LinAlgError:
                            continue
    return gs_summary

gr = grid_search(df_partitioned)

In [ ]:
import pickle

# pickle.dump(gr, open('./tmp/--', 'wb'))

In [ ]:
def grid_search_exp_smth(df_dict: Dict[str, pd.DataFrame]):
    from numpy.linalg import LinAlgError
    gs_summary = {
        'CO': [],
        'NOx': [],
        'NMHC': [],
        'O3': []
    }
    trend_types = ['add', 'mul']
    seasonal_types = ['add', 'mul']
    seasonal_periods = get_seasonal_periods(df_dict=df_dict)
    smoothing_levels = np.linspace(0.01, 0.75, 6)
    # smoothing_trend_space = np.linspace(0.01, 0.75, 6)
    # smoothing_seasonal_space
    for gas in df_dict.keys():
        for smoothing_level in smoothing_levels:
            for t in trend_types:
                for s in seasonal_types:
                    df = df_dict[gas]
                    try:
                        results = evaluate_params(df, algo='expsmooth', init_kws={'trend': t, 'seasonal': s, 'seasonal_periods': seasonal_periods[gas]}, fit_kws={'smoothing_level': smoothing_level})
                        gs_summary[gas].append({
                            'params': {
                                'trend': t,
                                'seasonal': s,
                                'smoothing_level': smoothing_level
                            },
                            'aic': results['aic'],
                            'train_rmse': results['train_rmse'],
                            'test_rmse': results['test_rmse']
                        })
                    except LinAlgError:
                        continue
    return gs_summary

ge = grid_search_exp_smth(df_partitioned)

In [ ]:
def evaluate_grid_search(gs_results, metric: Union['rmse', 'aic'] = 'rmse'):
    gs_best_params = {}
    for gas in gs_results.keys():
        res = gs_results[gas]
        index = np.argmin(list(map(lambda x: x['test_rmse'], res))) if metric == 'rmse' else np.argmin(list(map(lambda x: x['aic'], res)))
        print(gas)
        print(res[index])
        best_params = res[index]['orders']
        gs_best_params[gas] = {
            'params': best_params,
            'rmse': res[index]['test_rmse']
        }
    return gs_best_params

In [ ]:
exp_smth_best = evaluate_grid_search(ge, metric='rmse')

In [ ]:
sarima_best = evaluate_grid_search(gr, metric='rmse')

In [ ]:
def compare_algorithms_for_each_gas(sarima_gs: Dict[str, Dict], expsmth_gs: Dict[str, Dict]):
    best_algorithms = {} 
    for gas in sarima_gs.keys():
        current_sarima_gs = sarima_gs[gas]
        current_expsmth_gs = expsmth_gs[gas]

        sarima_rmse = current_sarima_gs['rmse']
        expsmth_rmse = current_expsmth_gs['rmse']

        if sarima_rmse < expsmth_rmse:
            model = SARIMAX,
            init = {
                'order': current_sarima_gs['params'][0],
                'seasonal_order': current_sarima_gs['params'][1],
                'trend': 'n'
            }
            fit = {}
        else:
            model = ExponentialSmoothing
            init = {
                'trend': current_expsmth_gs['params']['trend'],
                'seasonal': current_expsmth_gs['params']['seasonal']
            }
            fit = {
                'smoothing_level': current_expsmth_gs['params']['smoothing_level']
            }

        best_algorithms[gas] = {
            'model': model[0] if type(model) is tuple else model,
            'init': init,
            'fit': fit
        }
    return best_algorithms

algo_comparison = compare_algorithms_for_each_gas(sarima_best, exp_smth_best)
algo_comparison

In [ ]:
from sklearn.metrics import mean_squared_error

def evaluate_params_whole(df_dict: Dict[str, pd.DataFrame], models: Dict[str, Dict], test_size: float = 0.75):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', ConvergenceWarning)
        results = {}
        for gas in df_dict.keys():
            current_config = models[gas]
            algo = current_config['model']
            init_params = current_config['init']
            fit_params = current_config['fit']

            data = df_dict[gas]['Value']
            partition_index = int(data.shape[0] * test_size)

            train = data[:partition_index]
            test = data[partition_index:]
            
            model = algo(endog=train, **init_params).fit(**fit_params)
            
            train_pred = model.predict(start=0, end=partition_index - 1)
            test_pred = model.forecast(steps=data.shape[0] - partition_index)
            # test_pred = model.predict(start=partition_index, end=data.shape[0] - 1)

            train_err = mean_squared_error(train[7:], train_pred[7:], squared=False)
            test_err = mean_squared_error(test, test_pred, squared=False)

            results[gas] = {
                'aic': model.aic,
                'train_true': train,
                'train_pred': train_pred,
                'train_rmse': train_err,
                'test_true': test,
                'test_pred': test_pred,
                'test_rmse': test_err
            }
        return results

gg = evaluate_params_whole(df_partitioned, models=algo_comparison)

In [ ]:
def decompose(results: Dict[str, Dict], test_rmse_weight: float = 0.5):
    aic_values = []
    train_rmse_values = []
    test_rmse_values = []
    for gas in results.keys():
        aic_values.append(results[gas]['aic'])
        train_rmse_values.append(results[gas]['train_rmse'])
        test_rmse_values.append(results[gas]['test_rmse'])
    decomposed_results = {
        'mean_aic': sum(aic_values) / len(aic_values),
        'mean_train_rmse': sum(train_rmse_values) / len(train_rmse_values),
        'mean_test_rmse': sum(test_rmse_values) / len(test_rmse_values),
    }
    decomposed_results['overall_mean_rmse'] = (1 - test_rmse_weight) * decomposed_results['mean_train_rmse'] + test_rmse_weight * decomposed_results['mean_test_rmse']
    return decomposed_results

decompose(gg, test_rmse_weight=0.75)

In [ ]:
def plot_predictions(results: Dict[str, Dict]):
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12, 7))
    for i, gas in enumerate(results.keys()):
        current_results = results[gas]
        current_ax = ax[i // 2, i % 2]
        current_ax.plot(pd.concat(objs=(current_results['train_true'], current_results['test_true']), axis=0))
        current_ax.plot(current_results['train_pred'])
        current_ax.plot(current_results['test_pred'])
        current_ax.set_title(gas)

plot_predictions(gg)

In [ ]:
# def generate_combinations(orders: Tuple[Tuple[int], int, Tuple[int]], limit: int = None, empty: bool = False):
#     from more_itertools import powerset
#     from itertools import product
#     from numpy.random import shuffle

#     p = [v for v in powerset(orders[0]) if (len(v) > 0 or empty)]
#     q = [v for v in powerset(orders[2]) if (len(v) > 0 or empty)]

#     combinations = list(product(p, (orders[1],), q))
#     shuffle(combinations)

#     if limit is not None:
#         return combinations[:limit]
#     return combinations

# generate_combinations(((2, 3), 0, (1,)), empty=True)

In [ ]:
def get_test_df(file_path: str = './data/test.csv'):
    test_df = pd.read_csv(filepath_or_buffer=file_path, sep=',', header=0)
    test_df['Date'] = pd.to_datetime(test_df['Date'], format='%d/%m/%Y')
    test_df.set_index(keys='Date', inplace=True)
    return test_df.sort_index(ascending=True)

In [ ]:
def kaggle_submission(df_dict: Dict[str, pd.DataFrame], params: Dict[str, Dict], test_file: str, submission_file: str, training_size: float = 1.0):
    test_df = get_test_df()
    result_set = {}
    from statsmodels.tools.sm_exceptions import ConvergenceWarning
    from numpy.linalg import LinAlgError
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', ConvergenceWarning)
        for gas in df_dict.keys():
            df = df_dict[gas]['Value']
            df = df.iloc[int(1 - training_size * df.shape[0]):]
            target = test_df[test_df['Gas'] == gas]

            current_config = params[gas]
            algo = current_config['model']
            init_params = current_config['init']
            fit_params = current_config['fit']

            model = algo(endog=df, **init_params).fit(**fit_params)

            pred = model.forecast(steps=len(pd.date_range(df.index[-1], target.index[-1], freq='D'))).rename('Value')
            result = pd.merge(left=pred, right=target, left_index=True, right_index=True, how='inner')
            print(result.shape)
            result_set[gas] = result[['id', 'T', 'RH', 'Value']].set_index(keys='id')

    pred_df = pd.DataFrame(columns=result_set['CO'].columns)
    for gas in result_set.keys():
        pred_df = pd.concat(objs=(pred_df, result_set[gas]), axis=0)
    
    pred_df.sort_index(inplace=True)
    pred_df['Value'].to_csv(submission_file, sep=',', index_label='id')
    
    return pred_df.sort_index(ascending=True)

pred_df = kaggle_submission(df_partitioned, training_size=0.75, params=algo_comparison, test_file='./data/test.csv', submission_file='./out/sarima-and-expsmth-drop-outliers-inverse-0.75.csv')

In [ ]:
def plot_forecasts(df_dict: Dict[str, pd.DataFrame], submission_file: str):
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10, 8))
    test_df = get_test_df()
    pred_df = pd.read_csv(submission_file, index_col=0)
    for i, gas in enumerate(test_df['Gas'].unique()):
        df = df_dict[gas]
        current_ax = ax[i // 2, i % 2]
        target = test_df[test_df['Gas'] == gas]
        future = pd.merge(left=target, right=pred_df, left_on='id', right_index=True, how='inner')
        current_ax.plot(df['Value'])
        current_ax.plot(future['Value'])
        current_ax.set_title(gas)

plot_forecasts(df_partitioned, submission_file='./out/sarima-and-expsmth-drop-outliers-inverse-0.75.csv')

In [ ]:
def bar_forecasts():
    df = pd.read_csv('./out/sarima-and-expsmth-drop-outliers.csv', index_col=0)
    df.plot()

bar_forecasts()

# Part B > Clustering

## Import Exclusive Dependencies

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN, AffinityPropagation, AgglomerativeClustering

## Import Data

In [ ]:
df2 = pd.read_csv('./data/Mall_Customers.csv', index_col=0)
df2.head()

## Exploratory Data Analysis

In [ ]:
def rename_columns(df: pd.DataFrame):
    df_tmp = df.copy()
    df_tmp.rename(mapper={
        'Genre': 'Gender',
        'Annual Income (k$)': 'Annual Income',
        'Spending Score (1-100)': 'Spending Score'
    }, axis=1, inplace=True)
    df_tmp['Annual Income'] = df_tmp['Annual Income'] * 1000
    return df_tmp

df2 = rename_columns(df2)
df2

In [ ]:
def get_distribution_of_each_variable(df: pd.DataFrame):
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10, 8))
    for i, c in enumerate(df.columns):
        var = df[c]
        kde = var.dtype.kind in 'biufc'
        sns.histplot(var, kde=kde, ax=ax[i // 2, i % 2])

get_distribution_of_each_variable(df2)

In [ ]:
def get_distribution_by_gender(df: pd.DataFrame):
    fig, ax = plt.subplots(nrows=3, figsize=(8, 8))
    for i, c in enumerate(['Age', 'Annual Income', 'Spending Score']):
        var = df[c]
        sns.kdeplot(x=var, hue=df['Gender'], ax=ax[i])
    fig.tight_layout()

get_distribution_by_gender(df2)

# It seems that there are slightly more data on female customers than male ones

In [ ]:
def get_corr_heatmap(df: pd.DataFrame):
    sns.heatmap(df.corr(), annot=True, cmap='bone')

get_corr_heatmap(df2)

## Feature Engineering

In [ ]:
def standardize_variables(df: pd.DataFrame, cols: List[str]):
    df_tmp = df.copy()
    df_tmp[cols] = StandardScaler().fit_transform(df_tmp[cols])
    return df_tmp

df2_s = standardize_variables(df2, cols=['Age', 'Annual Income', 'Spending Score'])
df2_s

## Principal Component Analysis

In [ ]:
from typing import Union, List

def get_pca_results(df: pd.DataFrame, ignore_cols: Union[str, List[str]]):
    df_scaled = StandardScaler().fit_transform(X=df.drop(columns=ignore_cols))

    pca = PCA(n_components=df_scaled.shape[1]).fit(X=df_scaled)
    header = ['Eigenvalue', 'Explained Variance', 'Cumulative Explained Variance']
    header.extend(df.drop(columns=ignore_cols).columns.tolist())
    eigenvalues = pca.explained_variance_
    eigenvectors = pca.components_
    expl_var = pca.explained_variance_ratio_
    cum_expl_var = pca.explained_variance_ratio_.cumsum()
    pca_results = pd.DataFrame(
        data=np.hstack((
            eigenvalues.reshape(-1, 1),
            expl_var.reshape(-1, 1),
            cum_expl_var.reshape(-1, 1),
            eigenvectors
        )),
        columns=header,
        index=[f'PC {i + 1}' for i in range(df_scaled.shape[1])]
    )

    df_transformed = pd.DataFrame(
        data=pca.transform(df_scaled),
        index=df.index,
        columns=[f'PC {i + 1}' for i in range(df_scaled.shape[1])]
    )

    return pca_results, df_transformed

pca_results, df2_transformed = get_pca_results(df=df2, ignore_cols='Gender')
pca_results

In [ ]:
def scree_plot(df: pd.DataFrame, pca: pd.DataFrame):
    with sns.axes_style(style='darkgrid'):
        ax = sns.pointplot(data=pca, x=pca.index, y=pca['Eigenvalue'])
        ax.set(
            title='Scree Plot for PCA (df2)',
            ylim=(0, 1.4)
        )
        ax.annotate(text='As there is no elbow,\nno PC should be discarded', xy=(1.75, 1.2), ha='center')
        return ax

scree_plot(df2, pca_results)

## Visual Analysis - Algorithm

In [ ]:
# import plotly.express as px
# from sklearn.cluster import DBSCAN, OPTICS, AgglomerativeClustering, AffinityPropagation

# for mo in [KMeans(n_clusters=6), DBSCAN(eps=15, min_samples=15), OPTICS(max_eps=18)]:
#     colrs = mo.fit_predict(X=df2[['Annual Income (k$)', 'Age', 'Spending Score (1-100)']])
#     fig = px.scatter_3d(data_frame=df2, x='Annual Income (k$)', y='Age', z='Spending Score (1-100)', color=colrs, title=type(mo).__name__ + ' ' + str(round(silhouette_score(X=df2[['Annual Income (k$)', 'Age', 'Spending Score (1-100)']], labels=colrs), 2)), color_continuous_scale=px.colors.sequential.Rainbow)
#     fig.show()

In [ ]:
cluster_params = [4, 5]
fig, ax = plt.subplots(ncols=2, figsize=(10, 8))
for i, cl in enumerate(cluster_params):
    model = KMeans(n_clusters=cl).fit(X=df2_s[['Annual Income', 'Spending Score']])
    y_hat = model.predict(df2_s[['Annual Income', 'Spending Score']])
    sns.scatterplot(data=df2_s, x='Annual Income', y='Spending Score', hue=y_hat, ax=ax[i])
    print(f'Silhouette Score ({cl}):', silhouette_score(X=df2_s[['Annual Income', 'Spending Score']], labels=model.labels_, metric='euclidean'))

## Numerical Optimisation - Hyperparameter Selection

In [ ]:
def get_dist_score(df: pd.DataFrame, cols: List[str], clusters: int, plot: bool = False):
    model = KMeans(n_clusters=clusters, random_state=3).fit(X=df[cols])
    centers = model.cluster_centers_
    y_hat = model.predict(X=df[cols])
    
    df_tmp = pd.concat(objs=(df, pd.Series(data=y_hat, name='Class', index=df.index)), axis=1)
    means = []
    counts = []
    for i, c in enumerate(centers):
        class_i = df_tmp[df_tmp['Class'] == i][cols]
        distances = np.linalg.norm(class_i - centers[i, :], axis=1)
        means.append(distances.mean())
        counts.append(len(class_i))
    
    if plot == True:
        for i, m in enumerate(means):
            plt.plot(centers[i,0], centers[i,1], 'o', mfc='none', color='r', markersize=m * 5)
        sns.scatterplot(x=df['Annual Income'], y=df['Spending Score'], hue=y_hat)

    return np.std(means) / (sum(means) / len(means)), np.std(counts)
    # return np.std(means)

get_dist_score(df2, ['Age', 'Annual Income', 'Spending Score'], 6)

In [ ]:
def get_silhouette_score(df: pd.DataFrame, cols: List[str], clusters: int):
    X = df[cols]
    y_hat = KMeans(n_clusters=clusters).fit(X=X).predict(X=X)
    return silhouette_score(X=df[cols], labels=y_hat)

get_silhouette_score(df=df2, cols=['Age', 'Annual Income', 'Spending Score'], clusters=6)

In [ ]:
# type: ignore
from sklearn.metrics import silhouette_score
from more_itertools import powerset

def get_custom_score_plot(df: pd.DataFrame, cols: List[str], clusters: List[int] = list(range(2, 11)), ignore_cols: Union[str, List[str]] = None):
    distance_scores = [get_dist_score(df, cols, n)[0] for n in clusters]
    count_scores = [get_dist_score(df, cols, n)[1] for n in clusters]
    silhouette_scores = [get_silhouette_score(df, cols, n) for n in clusters]

    ax = sns.lineplot(x=clusters, y=silhouette_scores)
    ax2 = ax.twinx()
    ax3 = ax.twinx()
    sns.lineplot(x=clusters, y=distance_scores, color='g', ax=ax2)
    sns.lineplot(x=clusters, y=count_scores, color='r', ax=ax3)

get_custom_score_plot(df=df2, cols=['Age', 'Annual Income', 'Spending Score'])

In [ ]:
get_custom_score_plot(df=df2, cols=['Annual Income', 'Spending Score'])

In [ ]:
import plotly.express as px

def get_final_plot(df: pd.DataFrame):
    df_tmp = df.copy()
    X = df[['Age', 'Annual Income', 'Spending Score']]
    model = KMeans(n_clusters=6)
    model.fit(X=X)
    y_hat = model.predict(X=X)
    df_tmp['Class'] = y_hat
    fig = px.scatter_3d(data_frame=df_tmp, x='Age', y='Annual Income', z='Spending Score', color='Class')
    return fig

get_final_plot(df2)